# <a href="https://colab.research.google.com/github/biplobsd/RLabClone/blob/master/blackpeal_template.ipynb" target="_blank"><img src='https://biplobsd.github.io/RLabClone/img/black_pearl.png' height="50" alt="Black Pearl"/></a>

In [0]:
# ============================= FORM ============================= #
# @markdown ###Media Informations <a href="https://blackpearl.biz/" target="_blank">BlackPearl</a> <a href="https://myanimelist.net/" target="_blank">MAL</a> <a href="https://www.imdb.com/" target="_blank">IMDB</a> <a href="https://drive.google.com/drive/u/0/folders/0ANavpWDMk55-Uk9PVA" target="_blank">DRIVE</a> <a href="https://links.blackpearl.biz/" target="_blank">LinkSafe</a>
MODE = "IMDB"  # @param ['IMDB', 'MAL']
MEDIA_ID = ""  # @param {type:"string"}
PATH = ""  # @param {type:"string"}
LINK = ""  # @param {type:"string"}
SCREENSHOTS = ""  # @param {type:"string"}
TRAILER = ""  # @param {type:"string"}
# @markdown ---
# @markdown ###Forum Requirements
LIKE_REQUIREMENT = 10  # @param {type: "number"}
POST_REQUIREMENT = 10  # @param {type: "number"}
DAY_REQUIREMENT = 25  # @param {type: "number"}
THANK_REQUIREMENT = True  # @param {type:"boolean"}
# ============================= FORM ============================= #

import json, re, requests, sys  # pylint: disable=import-error
from os import path as _p
from IPython.display import clear_output  # pylint: disable=import-error
from IPython import get_ipython  # pylint: disable=import-error
from bs4 import BeautifulSoup  # pylint: disable=import-error


def checkAvailable(path_="", userPath=False):
    from os import path as _p

    if path_ == "":
        return False
    else:
        return (
            _p.exists(path_)
            if not userPath
            else _p.exists(f"/usr/local/sessionSettings/{path_}")
        )


def runSh(args, *, output=False, shell=False):
    import subprocess, shlex  # nosec

    if not shell:
        if output:
            proc = subprocess.Popen(  # nosec
                shlex.split(args), stdout=subprocess.PIPE, stderr=subprocess.STDOUT
            )
            while True:
                output = proc.stdout.readline()
                if output == b"" and proc.poll() is not None:
                    return
                if output:
                    print(output.decode("utf-8").strip())
        return subprocess.run(shlex.split(args)).returncode  # nosec
    else:
        if output:
            return (
                subprocess.run(
                    args,
                    shell=True,  # nosec
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                )
                .stdout.decode("utf-8")
                .strip()
            )
        return subprocess.run(args, shell=True).returncode  # nosec


if not checkAvailable("/usr/bin/mediainfo"):
    runSh("apt-get install mediainfo")

if not checkAvailable("/content/drive/Shared drives"):
    from google.colab import drive  # pylint: disable=import-error

    drive.mount("/content/drive")

# ================================================================ #

RQ_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:21.0) Gecko/20100101 Firefox/21.0"
}
API_KEY = "991109a2"


def getImdbID():
    if MEDIA_ID == "":
        print("Need IMDB ID")
        sys.exit()
    imdb_pattern = re.compile(
        "(tt\d{7,8})"  # pylint: disable=anomalous-backslash-in-string
    )
    return re.findall(imdb_pattern, MEDIA_ID)[0]


def getMalID():
    if MEDIA_ID == "":
        print("Need MAL ID")
        sys.exit()
    mal_pattern = re.compile(
        "(anime)?/(\d{1,7})/(.+)?"  # pylint: disable=anomalous-backslash-in-string
    )
    if re.match(r"\d+", MEDIA_ID):
        mal_id = MEDIA_ID
    else:
        mal_id = re.findall(mal_pattern, MEDIA_ID)[0][1]
    return mal_id


def getMediaId():
    if MODE != "MAL":
        return getImdbID()
    else:
        return getMalID()


def getStoryline(imdb_id):
    r = requests.get(
        f"https://www.imdb.com/title/{imdb_id}",
        headers=RQ_HEADERS,
        verify=False,  # nosec
    )
    clear_output()
    soup = BeautifulSoup(r.text, "html.parser")
    return soup.find_all("div", class_="inline canwrap")[0].find("span").text.lstrip()


def getAnimeInfo(anime_data, info):
    text = ""
    for i in range(len(anime_data[info])):
        text += anime_data[info][i]["name"] + ", "
    return text[:-2]


def generateParagraphHeadline(headline="Plot Summary"):
    return f"""[hr][/hr]
        [indent]\
            [size=6]\
                [font=tahoma]\
                    [color=rgb(84, 172, 210)]\
                        [b]\
                            {headline}\
                        [/b]\
                    [/color]\
                [/font]\
            [/size]\
        [/indent]"""


def getPropertyInfo(mediaInfo, prop="Duration"):
    propLen = len(prop)
    return next((x for x in mediaInfo if x[:propLen] == prop), None).rpartition(":")[2].strip()


TITLE = ""


def generateTitle():
    return _p.basename(PATH).rpartition(".")[0].replace(".", " ").replace("5 1", "5.1", 1).replace("6 1", "6.1", 1).replace("7 1", "7.1",1)


def generateMediaNfo(nfoFile):
    mediaInfo = nfoFile.readlines()
    del mediaInfo[1]
    size = getPropertyInfo(mediaInfo, "File size")
    duration = getPropertyInfo(mediaInfo, "Duration")
    global TITLE
    TITLE = f"{generateTitle()} [{size}] [{duration}]\n"
    mediaInfo[1] = f"{'Title name':<41}: {TITLE}\n"
    return "".join(mediaInfo)


def genTags():
    a = generateTitle().replace("2160p", "UHD 4K 2160p", 1).replace("x265", "x265 HVEC", 1).rpartition("-")
    b = f"{a[0]} {a[2]}".replace(" ", ",") + ","    
    print(TITLE)
    print(b)


# ================================================================ #


def outputBBTemplate(mediaId):
    youtubePattern = re.compile(
        "(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})"  # pylint: disable=anomalous-backslash-in-string
    )

    if LINK == "":
        print("You forgot putting in the link to download!")
        sys.exit()

    if PATH == "":
        mediainfoEnabled = False
    else:
        mediainfoEnabled = True
        runSh(f'mediainfo --Logfile=/root/.nfo "{PATH}"')
        with open("/root/.nfo", "r", encoding="utf-8") as nfo_file:
            mediaInfo = generateMediaNfo(nfo_file)

    if SCREENSHOTS == "":
        screenshotEnabled = False
    else:
        screenshotList = SCREENSHOTS.split(" ")
        screenshotBBCodes = ""
        for i in screenshotList:
            screenshotBBCodes += f"\n[img]{i}[/img]"
        screenshotEnabled = True

    # Fetch data
    if MODE != "MAL":
        imdb_id = mediaId
        imdb_url = (
            f"http://www.omdbapi.com/?i={imdb_id}&apikey={API_KEY}&r=json&plot=full"
        )
        imdb_data = json.loads(requests.get(imdb_url, verify=False).text)  # nosec
        poster = imdb_data["Poster"]
        mediaUrl = f"https://www.imdb.com/title/{imdb_id}"
        mediaTitle = f"{imdb_data['Title']} ({imdb_data['Year']})"
        imdbIcon = "[img height='16px']https://i.imgur.com/rcSipDw.png[/img]"
        ratings = imdb_data["imdbRating"]
        votes = imdb_data["imdbVotes"]
        imdbTrailerEnable = False
        if TRAILER != "":
            youtube_match = re.match(youtubePattern, TRAILER)
            if not youtube_match:
                clear_output()
                print("Your trailer link is invalid. Please only put in youtube link!")
                sys.exit()
            else:
                imdbTrailerEnable = True
                youtubeLink = youtube_match.group(6)
    else:
        if checkAvailable("/usr/local/lib/python3.6/dist-packages/jikanpy"):
            runSh("pip install git+git://github.com/AWConant/jikanpy.git")
        from jikanpy import Jikan  # pylint: disable=import-error

        anime_data = Jikan().anime(mediaId)

        poster = anime_data["image_url"]
        mediaUrl = anime_data["url"]
        mediaTitle = (
            f"{anime_data['title']} ({anime_data['type']} - {anime_data['premiered']})"
        )
        malIcon = "[img height='16px']https://i.imgur.com/YJgzM4V.png[/img]"
        ratings = anime_data["score"]
        votes = anime_data["scored_by"]
        imdbTrailerEnable = True
        youtubeLink = re.match(youtubePattern, str(anime_data["trailer_url"])).group(6)

    # Process data
    textDump = f"""\
        [center]
            [img width="350px"]{poster}[/img]
            [url={mediaUrl}][color=rgb(250, 197, 28)][font=tahoma][size=7]\
                [b]{mediaTitle}[/b]\
            [/size][/font][/color][/url]
            [url={mediaUrl}]\
                {imdbIcon if MODE != "MAL" else malIcon}\
            [/url]\
            [size=6]\
                [b] {ratings}[/b]/10 --- [img height="16px"]https://i.imgur.com/sEpKj3O.png[/img] {votes}\
            [/size]\
        [/center]
        {generateParagraphHeadline("Plot Summary" if MODE != "MAL" else "Synopsis")}\n
        {getStoryline(imdb_id) if MODE != "MAL" else anime_data['synopsis']}
    """

    if imdbTrailerEnable:
        textDump += f"""\
            {generateParagraphHeadline("Trailer")}\n
            [center]
                [media=youtube]\
                    {youtubeLink}
                [/media]\
            [/center]
        """

    if screenshotEnabled:
        textDump += f"""\
            {generateParagraphHeadline("Screenshots")}\n
            [spoiler='Expand Screenshots']
                {screenshotBBCodes}
            [/spoiler]
        """

    textDump += (
        f"""\
        {generateParagraphHeadline("Movie Info")}
        [LIST]
            [*][b][color=rgb(251, 160, 38)]Release Date :[/b][/color] {imdb_data['Released']}
            [*][b][color=rgb(251, 160, 38)]On Disk/Streaming :[/b][/color] {imdb_data['DVD'] if 'DVD' in imdb_data else "N/A"}
            [*][b][color=rgb(251, 160, 38)]Rating :[/b][/color] {imdb_data['Rated']}
            [*][b][color=rgb(251, 160, 38)]Genres :[/b][/color] {imdb_data['Genre']}
            [*][b][color=rgb(251, 160, 38)]Studio :[/b][/color] {imdb_data['Production'] if 'Production' in imdb_data else 'N/A'}
            [*][b][color=rgb(251, 160, 38)]Directed By :[/b][/color] {imdb_data['Director']}
            [*][b][color=rgb(251, 160, 38)]Written By :[/b][/color] {imdb_data['Writer']}
            [*][b][color=rgb(251, 160, 38)]Starring :[/b][/color] {imdb_data['Actors']}
            [*][b][color=rgb(251, 160, 38)]Awards :[/b][/color] {imdb_data['Awards']}
            [*][b][color=rgb(251, 160, 38)]Runtime :[/b][/color] {imdb_data['Runtime'] if 'Runtime' in imdb_data else 'N/A'}
            [*][b][color=rgb(251, 160, 38)]Box Office :[/b][/color] {imdb_data['BoxOffice'] if 'BoxOffice' in imdb_data else "N/A"}
        [/LIST]
    """
        if MODE != "MAL"
        else f"""\
        {generateParagraphHeadline("Anime Info")}
        [LIST]
            [*][b][color=rgb(251, 160, 38)]Premiered :[/b][/color] {anime_data['premiered']}
            [*][b][color=rgb(251, 160, 38)]Aired :[/b][/color] {anime_data['aired']['string']}
            [*][b][color=rgb(251, 160, 38)]Rating :[/b][/color] {anime_data['rating']}
            [*][b][color=rgb(251, 160, 38)]Genres :[/b][/color] {getAnimeInfo(anime_data, 'genres')}
            [*][b][color=rgb(251, 160, 38)]Studio :[/b][/color] {getAnimeInfo(anime_data, 'studios')}
            [*][b][color=rgb(251, 160, 38)]Producers :[/b][/color] {getAnimeInfo(anime_data, 'producers')}
            [*][b][color=rgb(251, 160, 38)]Type :[/b][/color] {anime_data['type']}
            [*][b][color=rgb(251, 160, 38)]Episodes :[/b][/color] {anime_data['episodes']}
            [*][b][color=rgb(251, 160, 38)]Duration :[/b][/color] {anime_data['duration']}
            [*][b][color=rgb(251, 160, 38)]Status :[/b][/color] {anime_data['status']}
            [*][b][color=rgb(251, 160, 38)]Source :[/b][/color] {anime_data['source']}
        [/LIST]
    """
    )

    if mediainfoEnabled:
        textDump += f"""\
        {generateParagraphHeadline("Media Info")}\n
            [spoiler='Expand Media Informations']\
                [code]\
                    {mediaInfo}
                [/code]\
            [/spoiler]\
        """

    textDump += f"""\
        {generateParagraphHeadline("Download Link")}
        {'[hidereact=1,2,4]' if THANK_REQUIREMENT else ''}
            {f'[hidereactscore={LIKE_REQUIREMENT}]' if LIKE_REQUIREMENT != 0 else ''}
                {f'[hideposts={POST_REQUIREMENT}]' if POST_REQUIREMENT != 0 else ''}
                    {f'[days={DAY_REQUIREMENT}]' if DAY_REQUIREMENT != 0 else ''}\
                        [downcloud]
                            {LINK}
                        [/downcloud]                        
                    {'[/days]' if DAY_REQUIREMENT != 0 else ''}
                {'[/hideposts]' if POST_REQUIREMENT != 0 else ''}
            {'[/hidereactscore]' if LIKE_REQUIREMENT != 0 else ''}
        {'[/hidereact]' if THANK_REQUIREMENT else ''}
    """

    return textDump


print(outputBBTemplate(getMediaId()))
genTags()

# <img src='https://biplobsd.github.io/RLabClone/img/title_utility.png' height="45" alt="Utility"/>

In [0]:
# ============================= FORM ============================= #
# @markdown <h3>⬅️ MKV Toolnix <a href="https://mkvtoolnix.download/docs.html" target="_blank">See Docs</a></h3>
MODE = "MEDIAINFO"  # @param ['MEDIAINFO', 'MKVPROPEDIT', 'MKVMERGE', 'MKVINFO']
FILE_PATH = ""  # @param {type:"string"}
OPTION = ""  # @param {type:"string"}
OUTPUT_FILE_PATH = ""  # @param {type:"string"}
# ============================= FORM ============================= #

from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec


def checkAvailable(path_="", userPath=False):
    from os import path as _p

    if path_ == "":
        return False
    else:
        return (
            _p.exists(path_)
            if not userPath
            else _p.exists(f"/usr/local/sessionSettings/{path_}")
        )


def runSh(args, *, output=False, shell=False):
    import subprocess, shlex  # nosec

    if not shell:
        if output:
            proc = subprocess.Popen(  # nosec
                shlex.split(args), stdout=subprocess.PIPE, stderr=subprocess.STDOUT
            )
            while True:
                output = proc.stdout.readline()
                if output == b"" and proc.poll() is not None:
                    return
                if output:
                    print(output.decode("utf-8").strip())
        return subprocess.run(shlex.split(args)).returncode  # nosec
    else:
        if output:
            return (
                subprocess.run(
                    args,
                    shell=True,  # nosec
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                )
                .stdout.decode("utf-8")
                .strip()
            )
        return subprocess.run(args, shell=True).returncode  # nosec


def installMkvTools():
    if checkAvailable("/etc/apt/sources.list.d/mkvtoolnix.download.list"):
        return
    if checkAvailable("/content/sample_data"):
        runSh("rm -rf /content/sample_data")
    with open("/etc/apt/sources.list.d/mkvtoolnix.download.list", "w+") as outFile:
        outFile.write("deb https://mkvtoolnix.download/ubuntu/ bionic main")
    runSh(
        "wget -q -O - https://mkvtoolnix.download/gpg-pub-moritzbunkus.txt | sudo apt-key add - && sudo apt-get install mkvtoolnix mkvtoolnix-gui",
        shell=True,  # nosec
    )
    if not checkAvailable("/usr/bin/mediainfo"):
        runSh("apt-get install mediainfo")


if not checkAvailable("/content/drive/Shared drives"):
    from google.colab import drive  # pylint: disable=import-error
    from IPython.display import clear_output

    try:
        drive.mount("/content/drive")
        clear_output(wait=True)
    except:
        clear_output()


def displayOutput(operationName="", color="#ce2121"):
    from IPython.display import HTML, display  # pylint: disable=import-error

    if color == "success":
        hColor = "#28a745"
        displayTxt = f"👍 Operation {operationName} has been successfully completed."
    elif color == "danger":
        hColor = "#dc3545"
        displayTxt = f"❌ Operation {operationName} has been errored."
    elif color == "info":
        hColor = "#17a2b8"
        displayTxt = f"👋 Operation {operationName} has some info."
    elif color == "warning":
        hColor = "#ffc107"
        displayTxt = f"⚠ Operation {operationName} has been warning."
    else:
        hColor = "#ffc107"
        displayTxt = f"{operationName} works."
    display(
        HTML(
            f"""
            <center>
                <h2 style="font-family:monospace;color:{hColor};">
                    {displayTxt}
                </h2>
                <br>
            </center>
            """
        )
    )


# ================================================================ #


def generateMediainfo():
    installMkvTools()
    if not FILE_PATH:
        print("Please enter path to media file from mounted drive.")
        return
    runSh("mkdir -m 777 -p /content/mediainfo")
    logFilePath = f"/content/mediainfo/{_p.basename(FILE_PATH).rpartition('.')[0]}.txt"

    if checkAvailable(logFilePath):
        runSh(f"rm -f {logFilePath}")
    runSh(f'mediainfo -f --LogFile="{logFilePath}" "{FILE_PATH}"')
    runSh(f"mkvmerge -i '{FILE_PATH}' >> '{logFilePath}'", shell=True)  # nosec
    with open(logFilePath, "r") as file:
        media = file.read()
        media = media.replace(f"{_p.dirname(FILE_PATH)}/", "")
    print(media)
    # !curl -s https://pastebin.com/raw/TApKLQfM -o /content/blackpearl.txt


def executeMKVTool():
    installMkvTools()
    if "/content/drive/Shared drives/" in FILE_PATH:
        print("File must be downloaded to local disk.")
        return
    if MODE == "MKVMERGE":
        if not OUTPUT_FILE_PATH:
            print("Must specify Output Path.")
            return
        runSh(
            f'mkvmerge -o "{OUTPUT_FILE_PATH}" {OPTION} "{FILE_PATH}" -v', output=True
        )
    elif MODE == "MKVPROPEDIT":
        runSh(f'mkvpropedit "{FILE_PATH}" {OPTION}', output=True)
    else:
        runSh(f'mkvinfo "{FILE_PATH}"', output=True)
    displayOutput(MODE, "success")


if MODE == "MEDIAINFO":
    generateMediainfo()
else:
    executeMKVTool()
